# Sentence Ebeddings et Phrases préfabriquées: tests préliminaires

Ce code a comme objectif de tester les capacités d'un modèle de sentence embedding à capturer le contenu sémantique d'un ensemble de phrases préfabriquées, puis d'examiner la possibilité d'améliorer ces capacités à travers un finetuning.

La procédure commence par initialiser le modèle "sentence-camembert-large" et une liste de phrases préfabriquées catégorisées pragmatiquement, puis calcule leurs embeddings sémantiques et évalue la similarité cosinus entre ces phrases et une phrase test. Ensuite, elle procède à un clustering (K-means) pour vérifier le regroupement sémantique des expressions. Enfin, le modèle est fine-tuné en utilisant des paires de phrases étiquetées comme similaires ou non-similaires selon leur catégorie pragmatique, dans le but d'affiner ses performances pour capturer les nuances sémantiques des expressions de surprise.

## 1. Préparation des bibliothèques, du modèle choisi et de la liste des phrases

### 1.1 Initialisation des modules nécessaires

In [36]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import random
from itertools import combinations
import pickle
import spacy
import pandas as pd
from sklearn.cluster import KMeans
import PyPDF2
import re

### 1.2.  Initialisation du modèle sentence-camembert-large

In [2]:
 model = SentenceTransformer("Lajavaness/sentence-camembert-large")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### 1.3. Préparation d'une liste de PPI représentant différents schémas pragma-sémantiques (Tutin & Grossmann, 2023)

In [3]:
sentences =     [
        # Surprise-plaisanterie
        "c’est une blague","c'est une farce ?","tu rigoles","tu plaisantes ?","tu déconnes?","sans déconner","sans rire?","sans blague ?",
        # Surprise-mutisme
        "ça m’en bouche un coin","ça me laisse sans voix","je sais pas quoi dire","j’en reste bouche bée","j’ai du mal à m’en remettre",
        # Surprise-réaction physique et mentale
        "je suis sous le choc","je suis sur le cul","les bras m’en tombent",
        # Surprise-caractère hors norme
        "c’est fou","c’est dingue", "c’est bizarre comme c’est curieux",
        # Surprise-caractère improbable
        "c’est incroyable","c’est pas croyable", "c’est pas Dieu possible","c’est pas vrai",
        # Surprise-absence d’évidence
        "comment est-ce possible","j’en crois pas mes oreilles","j’en crois pas mes yeux",
        # Surprise-caractère irréel
        "je rêve","j’hallucine","on croit rêver",
        # Surprise-fait surprenant
        "ça m’étonne","ça me stupéfie",
        # Surprise-expérienceur surpris
        "je suis baba","je suis ébahi","je suis stupéfait",
    ]

## 2. Test du modèle sur les phrases choisies

### 2.1. Création des plongements sémantiques des phrases avec le modèle sentence-camembert-large

In [4]:
embeddings_1 = model.encode(sentences)

### 2.2. Création des plongements sémantiques d'une phrases supplémentaire pour le test de similarité

In [5]:
sentence =  ["est-ce que c’est une blague"]
embeddings_2 = model.encode(sentence)

### 2.3. Test du modèle sur les phrases choisies: similarité cosinus et clustering

#### 2.3.1. Fonction réutilisable pour le calcul de similarité cosinus

In [6]:

def get_similarities(sentences_,embeddings_1_,embeddings_2_):
    similarities = model.similarity(embeddings_1_,embeddings_2_)
    scores = []
    for score in similarities.tolist():
        for s in score:
           scores.append(f"{s:.3f}")

    df_similarité = pd.DataFrame({"Phrases préfabriquées":sentences_,"Score de similarité":scores})

    return df_similarité.sort_values("Score de similarité",ascending=False)




#### 2.3.2. Calcul de similarité entre la phrase chosie et la liste des phrases


In [7]:
get_similarities(sentences,embeddings_1,embeddings_2)

,Phrases préfabriquées,Score de similarité
0,c’est une blague,0.850
7,sans blague ?,0.792
6,sans rire?,0.682
1,c'est une farce ?,0.672
3,tu plaisantes ?,0.646
5,sans déconner,0.617
2,tu rigoles,0.572
4,tu déconnes?,0.452
22,c’est pas vrai,0.376
8,ça m’en bouche un coin,0.298


#### 2.3.3. Création d'une fonction réutilisable pour le clustering des phrases par le modèle KMeans

In [8]:
def cluster_sentences(embeddings_,sentences_):
    KMeans_model = KMeans(n_clusters = 9,random_state = 0)
    KMeans_model.fit(embeddings_)

    kmeans_clusters = KMeans_model.labels_
    df_kmeans = pd.DataFrame({"Phrases":sentences_,"cluster":kmeans_clusters})

    return df_kmeans.sort_values("cluster")


#### 2.3.4. Clustering des phrases

In [9]:

cluster_sentences(embeddings_1,sentences)

,Phrases,cluster
18,c’est bizarre comme c’est curieux,0
27,j’hallucine,1
21,c’est pas Dieu possible,1
23,comment est-ce possible,1
19,c’est incroyable,1
17,c’est dingue,1
16,c’est fou,1
20,c’est pas croyable,1
11,j’en reste bouche bée,2
9,ça me laisse sans voix,2


## 5. Tentative d'amélioration du modèle à travers un petit finetuning sur des paires de phrases

### 5.1 Function pour la préparation des données d'apprentissage au format compatible avec Sentence Transformers

In [10]:
def create_training_pairs(clusters, num_positive_per_cluster=3, num_negative_per_sentence=2):

    train_pairs = []

    all_sentences = []
    cluster_labels = []
    for cluster_id, sentences in clusters.items():
        for sent in sentences:
            all_sentences.append(sent)
            cluster_labels.append(cluster_id)

    # Create positive pairs (same cluster)
    for cluster_id, sentences in clusters.items():
        if len(sentences) >= 2:
            # Generate combinations of sentences within the same cluster
            same_cluster_pairs = list(combinations(sentences, 2))
            # Sample positive pairs
            sampled_positives = random.sample(
                same_cluster_pairs,
                min(num_positive_per_cluster * len(sentences), len(same_cluster_pairs))
            )
            for sent1, sent2 in sampled_positives:
                train_pairs.append((sent1, sent2, 1.0))  # High similarity

    # Create negative pairs (different clusters)
    for i, (sent1, cluster1) in enumerate(zip(all_sentences, cluster_labels)):
        # Find sentences from different clusters
        different_cluster_sentences = [
            (sent2, cluster2) for j, (sent2, cluster2) in enumerate(zip(all_sentences, cluster_labels))
            if i != j and cluster1 != cluster2
        ]

        if different_cluster_sentences:
            # Sample negative pairs
            sampled_negatives = random.sample(
                different_cluster_sentences,
                min(num_negative_per_sentence, len(different_cluster_sentences))
            )
            for sent2, cluster2 in sampled_negatives:
                train_pairs.append((sent1, sent2, 0.0))  # Low similarity

    return train_pairs

### 5.2. Préparation d'un petit training set

In [12]:

# Préparation des catégories pragma-sémantiques des PPI
clusters = {
    1: ["c'est une blague", "c'est une farce ?", "tu rigoles", "tu plaisantes ?",
        "tu déconnes?", "sans déconner", "sans rire?", "sans blague ?","est-ce que c'est une blague?"],

    2: ["ça m'en bouche un coin", "ça me laisse sans voix", "je sais pas quoi dire",
        "j'en reste bouche bée"],

    3: ["j'ai du mal à m'en remettre", "je suis sous le choc", "je suis sur le cul",
        "les bras m'en tombent"],

    4: ["c'est fou", "c'est dingue", "c'est bizarre comme c'est curieux"],

    5: ["c'est incroyable", "c'est pas croyable", "c'est pas Dieu possible", "c'est pas vrai"],

    6: ["comment est-ce possible", "j'en crois pas mes oreilles", "j'en crois pas mes yeux"],

    7: ["je rêve", "j'hallucine", "on croit rêver"],

    8: ["ça m'étonne", "ça me stupéfie"],

    9: ["je suis baba", "je suis ébahi", "je suis stupéfait"]
}



# Création des training pairs compatibles avec la bibliothèque sentence transformers
train_pairs = create_training_pairs(clusters, num_positive_per_cluster=2, num_negative_per_sentence=3)

print(f"Created {len(train_pairs)} training pairs")
print("Sample positive pairs:")
for pair in train_pairs[:5]:
    if pair[2] == 1.0:
        print(f"  {pair[0]} || {pair[1]} -> {pair[2]}")
print("Sample negative pairs:")
for pair in train_pairs[-5:]:
    if pair[2] == 0.0:
        print(f"  {pair[0]} || {pair[1]} -> {pair[2]}")

# Convert to InputExample format
train_examples = []
for sent1, sent2, label in train_pairs:
    train_examples.append(InputExample(texts=[sent1, sent2], label=label))

# Create DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)

# Use CosineSimilarityLoss for fine-tuning
train_loss = losses.CosineSimilarityLoss(model)


Created 154 training pairs
Sample positive pairs:
  c'est une blague || sans blague ? -> 1.0
  c'est une farce ? || tu déconnes? -> 1.0
  c'est une blague || sans rire? -> 1.0
  c'est une blague || tu rigoles -> 1.0
  tu rigoles || sans blague ? -> 1.0
Sample negative pairs:
  je suis ébahi || c'est pas vrai -> 0.0
  je suis ébahi || je sais pas quoi dire -> 0.0
  je suis stupéfait || tu rigoles -> 0.0
  je suis stupéfait || les bras m'en tombent -> 0.0
  je suis stupéfait || c'est pas vrai -> 0.0


### 5.3 Finetuning rapide du modèle

In [18]:
# Training parameters
epochs = 3
warmup_steps = 15

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='./surprise_semantic_model',
    show_progress_bar=True
)

Step,Training Loss


### 5.4. Test du modèle après finetuning

#### 5.4.1. Similarité

In [20]:
#réencodage des phrases avec le modèle finetuné
embeddings_1 = model.encode(sentences)
embeddings_2 = model.encode(sentence)


get_similarities(sentences,embeddings_1,embeddings_2).head(20)

,Phrases préfabriquées,Score de similarité
1,c'est une farce ?,0.997
0,c’est une blague,0.996
3,tu plaisantes ?,0.992
6,sans rire?,0.992
7,sans blague ?,0.991
2,tu rigoles,0.988
4,tu déconnes?,0.988
5,sans déconner,0.974
25,j’en crois pas mes yeux,0.052
24,j’en crois pas mes oreilles,0.046


#### 5.4.2. Clustering

In [21]:
cluster_sentences(embeddings_1,sentences)

,Phrases,cluster
17,c’est dingue,0
18,c’est bizarre comme c’est curieux,0
16,c’est fou,0
12,j’ai du mal à m’en remettre,1
14,je suis sur le cul,1
13,je suis sous le choc,1
15,les bras m’en tombent,1
31,je suis baba,2
33,je suis stupéfait,2
32,je suis ébahi,2


### 6. Test sur le roman ***Les Racines du Mal***

#### 6.1. fonctions pour la lecture, prétraitement et embedding du PDF du roman

In [38]:
nlp = spacy.load("fr_core_news_sm")

def preprocess_text(text, paragraphs=False):
    # Nettoyage de base
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    nlp.max_length = 2000000

    doc = nlp(text)
    phrases = []
    count = 0


        # Original sentence segmentation
    for sent in doc.sents:
        count += 1
        #print(f"processed: {count} sentences")
        phrases.append(sent.text.strip())

    return phrases


def load_from_pdf(pdf,paragraphs=False,preprocess=True):
    print("reading from pdf...")
    with open(pdf+".pdf", 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        result = " ".join([page.extract_text() for page in reader.pages])
        print("Processing text...")
        if preprocess:
            result = preprocess_text(result,paragraphs)
    return result


def embnsave(src_pdf,dest_emb_pkl,dest_text_pkl,paragraphs=False,from_txt=False):
    if from_txt:
        sentences = load_from_txt(src_pdf)
    else:
        sentences = load_from_pdf(src_pdf,paragraphs)
    print("Embedding in progress...")
    embeddings = model.encode(sentences)
        #save embeddings
    with open(dest_emb_pkl, 'wb') as f:
        pickle.dump(embeddings, f)
        #save text:
    with open(dest_text_pkl, 'wb') as f:
        pickle.dump(sentences, f)
    return embeddings,sentences


def load_embs(src_emb_pkl,src_text_pkl):
    with open(src_emb_pkl, 'rb') as f:
        embeddings = pickle.load(f)
    with open(src_text_pkl, 'rb') as f:
        sentences = pickle.load(f)
    return embeddings,sentences


#### 6.2. Création des plongements du roman ***Les Racines du mal*** de Maurice Dantec

In [39]:

%%time
embeddings_racines_du_mal,sentences_racines_du_mal = embnsave("/content/racines_du_mal",
                                                              "racines_dumal_embeddings.pkl",
                                                              "racines_dumal_text.pkl")

reading from pdf...
Processing text...
Embedding in progress...
CPU times: user 2min 24s, sys: 4.46 s, total: 2min 29s
Wall time: 2min 30s


Le processus a duré 2min 30s sur un GPU T4 de 15GB (Google colab notebook) pour un roman de ~700 pages

#### 6.3. Test de similarité cosinus avec la phrase "est-ce que c'est une blague?"

In [40]:
%%time
get_similarities(sentences_racines_du_mal,
                 embeddings_racines_du_mal,
                 embeddings_2).head(30)

CPU times: user 64.1 ms, sys: 60 ms, total: 124 ms
Wall time: 126 ms


,Phrases préfabriquées,Score de similarité
5099,C'est une blague ?,0.996
7484,Non mais vous rigolez ?,0.992
4052,"Non, mais vous plaisantez ?",0.991
11422,Tu déconnes ?,0.988
3179,"Ah, arrête de déconner...",0.978
4968,Je me suis mis à rigoler.,0.977
9551,"Il s'est simplement marré, en guise de réponse.",0.972
10697,LOLITAS IN,0.968
8947,S'agissait de pas déconner.,0.964
5756,Un petit éclat de rire.,0.964
